In [1]:
#Imports
import sys

import pandas as pd # type: ignore
import numpy as np # type: ignore

import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore

import networkx as nx # type: ignore
import graphviz # type: ignore

import statsmodels.formula.api as smf # type: ignore
import statsmodels.api # type: ignore
from statsmodels.miscmodels.ordinal_model import OrderedModel # type: ignore

from dowhy import CausalModel, gcm # type: ignore

from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.neighbors import NearestNeighbors # type: ignore

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

/Users/annanatasha/Documents/DSO 585 Consulting/causal-attribution-analysis/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("../data/processed/data.csv")
df.shape

## Propensity Score Matching (Outcome is payment_value)

In [ ]:
# Define covariates, treatment, and outcome
covariates = ['Product_category', 'season', 'Product_size', 'freight_value', 'distance_km']
treatment = 'is_delivery_late'
outcome = 'payment_value'

In [ ]:
# Step 1: Build the causal model
# We specify the treatment, outcome, and confounders in the model.
model = CausalModel(
    data=df,
    treatment=treatment,
    outcome=outcome,
    common_causes=covariates
)

# Step 2: View the graphical model (optional)
# This visualizes the causal structure and how variables are related.
model.view_model()

In [ ]:
# Step 3: Identify the causal effect
# Specify how to estimate the causal effect. Here we use matching with propensity scores.
identified_estimand = model.identify_effect()
print(identified_estimand)

In [ ]:
# Step 4: Estimate the treatment effect using propensity score matching
# This will estimate the causal effect using Propensity Score Matching (PSM).
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.propensity_score_matching"
)

In [ ]:
# Step 5: View the results
# This will show the average treatment effect (ATE) of late deliveries on review scores.
print(estimate)

In [ ]:
# Display the causal estimate, confidence intervals, and standard error
print("Causal Estimate (ATE):", estimate.value)

In [ ]:
print("Confidence Interval:", estimate.get_confidence_intervals())

In [ ]:
#print("Confidence Interval:", estimate.get_confidence_intervals())
print("Standard Error:", estimate.get_standard_error())

In [ ]:
# Add an unobserved confounder and see how the treatment effect changes
refutation_unobserved_confounding = model.refute_estimate(
    identified_estimand,
    estimate,
    method_name="add_unobserved_common_cause"
)

# Display the refutation results
print(refutation_unobserved_confounding)

In [ ]:
# Estimate ATT using DoWhy
estimate_ATT = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.propensity_score_matching",
    target_units="att"  # Specify that we are interested in the ATT (treated units)
)

# Display the ATT
print("ATT (Average Treatment effect on the Treated):", estimate_ATT.value)

In [ ]:
# Estimate ATC using DoWhy
estimate_ATC = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.propensity_score_matching",
    target_units="atc"  # Specify that we are interested in the ATC (control units)
    #ATC is where is_delivery_late = 0, so what would the outcome have been for those who didn't experience a delayed delivery, if they had
)

# Display the ATC
print("ATC (Average Treatment effect on the Controls):", estimate_ATC.value)